In [1]:
def thumb_img1():
    image = soup.select_one('#sectionView > div > div.thumb > img')
    return image['src']

def prod_name():
    return soup.select_one('#sectionView > div > p.goods_name > strong').string.strip()
    
def prod_summary():
    return soup.select_one('#sectionView > div > p.goods_name > span.short_desc').string.strip()

def prod_price():
    return soup.select_one(
        '#sectionView > div > p.goods_price > span.position > span.dc > span.dc_price').get_text()
            
def prod_discount_rate():
     return soup.select_one(
                '#sectionView > div > p.goods_price > span.position > span.dc > span.dc_percent').string.strip()
    
def prod_info():
    unit, amount, package, made_in = '', '', '', ''
    goods_info = soup.select_one('#sectionView > div > div.goods_info')        
    for dl in goods_info:
        try:
            dt = dl.select_one('dt.tit')
            dd = dl.select_one('dd.desc')
            if dt.text == '판매단위':
                unit = dd.text
                continue
            elif dt.text == '중량/용량':
                amount = dd.text
                continue
            elif dt.text == '원산지':
                made_in = dd.text.strip()
                continue
            elif dt.text == '포장타입':
                for d in dd:
                    package = d.string.strip()
                    break
                continue
        except:
            pass
    return unit, amount, package, made_in

def prod_img2():
    img2 = soup.select_one(
                '#goods-infomation > div.goods_wrap > div.goods_intro > div.pic > img')
    return 'https:' + img2['src']

def prod_desc():
    return soup.select_one(
                '#goods-infomation > div.goods_wrap > div.goods_intro > div.context.last > p').string

def prod_img4():
    image = soup.select_one('#goods-infomation > div.goods_wrap > div.goods_point > div > div > img')
    image4 = 'https:' + image['src']
    return image4

def prod_img3():
    img3 = soup.select_one(
                '#goods_pi > p > img')
    return 'https:' + img3['src']



def prod_options():
    name_list, price_list = [], []
    option_select = driver.find_element_by_class_name("txt_select").click()
    option_url = driver.current_url
    driver.get(option_url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    options = soup.select_one('#cartPut > div.cart_option.cart_type2 > div > div.in_option > div.list_goods > div > div > ul')
    for li in options:
        for span in li.a:
            option_name = span.text.strip()
            option_name = re.sub('[(품절)]', '',option_name)
            break
        option_price = li.find("span", class_="dc_price").text.strip()
        name_list.append(option_name)
        price_list.append(option_price)
    return name_list, price_list 



In [21]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import json
import time
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# options = webdriver.ChromeOptions()
# options.add_argument('headless')
# options.add_argument("disable-gpu")

CNT = [0, 9, 9, 9, 9, 9, 9, 9,9, 9, 9, 9, 9, 9, 9, 9, 9]


driver = webdriver.Chrome('/home/eunbi/Desktop/Study/크롤링/chromedriver')
time.sleep(2)
driver.get('https://www.kurly.com/m2/index.php')
time.sleep(2)

driver.find_element_by_css_selector('#userMenu > li:nth-child(2) > a').click()
time.sleep(2) 


for i in range(13, 16):
    # 전체 카테고리 드롭다운 버튼 클릭
    driver.find_element_by_css_selector(
    '#userMenu > li:nth-child(2) > a').click()
    time.sleep(2)
    # 카테고리 선택
    driver.find_element_by_css_selector(
        f'#gnbMenu > ul > li:nth-child({i}) > a').click()
    time.sleep(2)
    
    driver.find_element_by_css_selector(f'#gnbMenu > ul > li:nth-child({i}) > div > ul > li:nth-child(1) > a').click()
    driver.execute_script('window.scrollTo(0, 500);') 
    # 서브 카테고리 탭 선택
    for s in range(2, CNT[i]+2): 
        
        WebDriverWait(driver, 10).until(EC.presence_of_element_located(
            (By.CSS_SELECTOR,f'#lnbMenu > div > ul > li:nth-child({s}) > a'))).click()

        time.sleep(2)
        subcategory_name =   WebDriverWait(driver, 10).until(EC.presence_of_element_located(
            (By.CSS_SELECTOR, f'#lnbMenu > div > ul > li:nth-child({s}) > a')))
        sub_name = subcategory_name.text
        time.sleep(2)
        subcategory_url = driver.current_url
        driver.execute_script('window.scrollTo(0, 1200);') 
        with open(f'{sub_name}.json', 'a+') as file:
        
            # 제품 상세 화면 클릭
            for g in range(1, 17):
                time.sleep(5)
                try:
                    WebDriverWait(driver, 20).until(EC.presence_of_element_located(
                 (By.CSS_SELECTOR, f'#goodsList > div.list_goods > ul > li:nth-child({g}) > div > a'))).click()
                except:
                    break
                time.sleep(2)
                # soup 설정
                link = driver.current_url
                driver.get(link)
                time.sleep(2)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                time.sleep(2)
                # 썸네일
#                 img1 = thumb_img1()

                # 상품명
                name = prod_name()

                # 짧은설명
                try:
                    summary = prod_summary()
                except:
                    summary = ''
                # 가격
                price = prod_price()

                ## 할인
                try:
                    discount_rate = prod_discount_rate()
                except:
                    discount_rate = 0

                unit, amount, package, made_in = prod_info()

                # 큰 이미지
                try:
                    img2 = prod_img2()
                except:
                    img2 = ''
                # 상품설명
                try:    
                    description = prod_desc()
                except:
                    description = ''
                
                try:
                    img4 = prod_img4()
                except:
                    img4 = ''
            
                # 상품이미지
                try:
                    img3 = prod_img3()
                    time.sleep(1)
                except:
                    img3 = ''
                    
                # options
                try:
                    option_name_list, option_price_list = prod_options()
                except:
                    option_name_list, option_price_list = [], []    
                print('----------------------------------------------------------------------')
                
                # 서브카테고리 메뉴로 가기
                driver.get(subcategory_url) 
                time.sleep(5)
                driver.execute_script(f'window.scrollTo(0, {1200 + 350*g});') 
                time.sleep(5)
                
                print(img4, name)
                time.sleep(3)

                data = {}
                
                data['name'] = name
                data['img4'] = img4

                file.seek(0)
                d = file.read(100)
                if len(d) > 0:
                    file.write(',\n')

                json.dump(data, file, ensure_ascii=False)


----------------------------------------------------------------------
https://img-cf.kurly.com/shop/data/goodsview/20200401/gv00000088319_2.jpg [에콜그린] 버블리콘 수세미
----------------------------------------------------------------------
https://img-cf.kurly.com/shop/data/goodsview/20200203/gv00000080077_2.jpg [밧드야] 스테인리스 키친랙
----------------------------------------------------------------------
https://img-cf.kurly.com/shop/data/goodsview/20200203/gv20000080034_2.jpg [밧드야] 스테인리스 음식물 쓰레기통 2종
----------------------------------------------------------------------
https://img-cf.kurly.com/shop/data/goodsview/20200203/gv00000080052_2.jpg [밧드야] 스테인리스 쌀통 10kg
----------------------------------------------------------------------
https://img-cf.kurly.com/shop/data/goodsview/20200102/gv20000076295_2.jpg [코디] 행주 대신 빨아쓰는 행주타월 50매 3롤
----------------------------------------------------------------------
https://img-cf.kurly.com/shop/data/goodsview/20191210/gv00000072238_2.jpg [에코후레쉬] 냉장고 냄새 탈취제
-------

----------------------------------------------------------------------
https://img-cf.kurly.com/shop/data/goodsview/20200109/gv30000077236_2.jpg [Kurly] 글라스락 간편 쿡 용기
----------------------------------------------------------------------
https://img-cf.kurly.com/shop/data/goodsview/20191209/gv40000071988_2.jpg [보스카] 세미 소프트 치즈 나이프
----------------------------------------------------------------------
https://img-cf.kurly.com/shop/data/goodsview/20191121/gv40000069049_2.jpg [광주요] 백자 음양각 돋을문 다관 세트
----------------------------------------------------------------------
https://img-cf.kurly.com/shop/data/goodsview/20191121/gv40000069042_2.jpg [광주요] 미각 설백 복주머니 다관 세트
----------------------------------------------------------------------
https://img-cf.kurly.com/shop/data/goodsview/20191121/gv00000069033_2.jpg [광주요] 한결 직사각 접시 2종 (연청)
----------------------------------------------------------------------
https://img-cf.kurly.com/shop/data/goodsview/20191121/gv30000069001_2.jpg [광주요] 한결 도너츠 플레이트 3

TimeoutException: Message: 
